#M.Lytova, M.Spanner, I.Tamblyn. Deep learning and high harmonic generation (2020)
##Codes for Section II.A : Surrogate model 

##Headers and constants

In [ ]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Dropout
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam, Nadam
from keras import objectives
from keras.losses import mean_squared_error
from keras.callbacks import TensorBoard
import argparse
import matplotlib.pyplot as plt
import time

In [ ]:
PI = 3.14159265359

x_n_points = 512   # number of nodes in input layer
x_n = np.linspace(0, 100, x_n_points) 

batch_size = 128

n_train = 500000   
n_test = 50000

##Training set generation

In [ ]:
# Training set
w0_train = np.random.rand(n_train,1)*0.5+0.5    # vector of random frequencies in [0.5, 1]
y_train = np.sin(w0_train*x_n)

##Testing set generation

In [ ]:
# Testing set
w0_test = np.random.rand(n_test,1)*0.5+0.5   # vector of random frequencies in [0.5, 1]
y_test = (np.sin(w0_test*x_n))

##Model

1$->$ 16 $->$ 64 $->$ 128 $->$ 512

In [ ]:
inputs = Input(shape = (1,))

x = Dense(16, activation='tanh')(inputs)
x = Dense(64, activation='tanh')(x)
x = Dense(128, activation='tanh')(x)

outputs = Dense(x_n_points, activation='tanh')(x)

ModelGen = Model(inputs, outputs)
opt = Nadam(lr=0.0001)
ModelGen.compile(optimizer=opt, loss='mean_squared_error') 

print(ModelGen.summary())

##Training

In [ ]:
tic = time.perf_counter()

history = ModelGen.fit(w0_train, y_train, 
                epochs=200,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(w0_test, y_test))

toc = time.perf_counter()
print(f"Execution time {toc - tic:0.4f} seconds")

def plot_losses():
    plt.figure(figsize=(8,4))
    plt.plot(np.log10(history.history['loss']),color='blue')
    plt.plot(np.log10(history.history['val_loss']),color='red')
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()

##Training and validation losses

In [ ]:
plot_losses()

##Prediction

In [ ]:
prediction = ModelGen.predict(w0_test)

##Function to draw the test and predicted examples

In [ ]:
def plot_examples(i1, i2):    
    fig = plt.subplots(2,1,figsize=(12,8),constrained_layout=False)
    plt.suptitle('Examples: Test points and prediction', fontsize=16)
    plt.subplot(211)
    plt.title('w0 = ' + str(round(w0_test[i1,0],2)), fontsize=16)
    plt.scatter(x_n, y_test[i1], color="blue", s = 1)
    plt.plot(x_n, prediction[i1], color="red", linewidth = 1)
    plt.grid()
    plt.subplot(212)
    plt.title('w0 = ' + str(round(w0_test[i2,0],2)), fontsize=16)
    plt.scatter(x_n, y_test[i2], color="blue", s = 1)
    plt.plot(x_n, prediction[i2], color="red", linewidth = 1)  
    plt.xlabel('$t$, fs', fontsize=16)
    plt.grid()  
    plt.show() 
    plt.close()    

## Comparison of arbitrary $y_{test}$ (blue) and $y_{predicted}$ (red)

In [ ]:
i_show1 = np.random.randint(0, n_test-1)
i_show2 = np.random.randint(0, n_test-1)

plot_examples(i_show1, i_show2)